<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c825757e0f89c8833b696118eb172200d052d1248dea59b4ffe9c191f0fe0a05
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-29 14:43:01
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: 56.10 K (0.41%)
Current PnL: -22.82 L (-15.35%)
CY Booked + Current PnL: -11.38 L (-7.65%)
-------------------
Total profit:  97.58 K
Total loss:  -23.79 L
-------------------
Total Booked + Current PnL: 15.60 L (12.82%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.36%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.68 L (64.84%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,93.38,36.0,M-SC,2.45,83303.0,-17474.0,17569.0,0.15,-17.34,21.09,0.09,245.0,-0.99,0.61,9.44,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,47.0,H-MC,3.63,189390.0,24720.0,18617.0,0.20,15.01,9.83,26.31,89.0,1.33,1.39,24.19,X40N,NTT,AC
50,NATIONALUM,244.55,-45.32,53.0,H-MC,11.90,108857.0,8353.0,20759.0,2.40,8.31,19.07,28.96,79.0,0.40,0.80,45.10,MH,ATH,METALS
42,ITC,452.00,-38.77,45.0,H-LC,2.15,198094.0,-2044.0,22939.0,1.25,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG
84,VOLTAS,1530.00,-1.92,47.0,H-MC,2.88,205650.0,13908.0,23855.0,2.30,7.25,11.60,19.69,99.0,0.58,1.50,14.54,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.0,-180.68,74.0,M-SC,6.74,92964.0,-9246.0,102632.0,12.42,-9.05,110.40,91.37,208.0,-0.09,0.68,50.91,XY25,NTT,FINANCE
56,RELAXO,1176.0,-36.49,47.0,H-SC,2.01,86630.0,-58530.0,134458.0,7.10,-40.32,155.21,52.31,136.0,-0.44,0.63,15.96,X40N,NTT,FOOTWEAR
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,10.45,124686.0,-17793.0,37319.0,4.53,-12.49,29.93,13.70,163.0,-0.48,0.91,47.16,OX40N,NTT,BANKS
53,QUESS,986.0,-42.15,46.0,M-SC,38.67,57597.0,-7409.0,158334.0,4.03,-11.40,274.90,232.18,198.0,-0.05,0.42,4.03,XY24,NTT,MISC
10,BANDHANBNK,400.0,-13.99,46.0,H-SC,4.16,215928.0,-62631.0,320070.0,3.95,-22.48,148.23,92.42,151.0,-0.20,1.58,24.79,XY24,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,HATHWAY,31.40,860.14,26.0,H-SC,2.02,82560.0,-26160.0,105842.0,-3.03,-24.06,128.20,73.29,161.0,-0.25,0.60,10.43,XR,NTT,ENTERTAINMENT
85,WHIRLPOOL,2270.00,-46.60,26.0,M-SC,1.31,90352.0,-1146.0,79898.0,-2.26,-1.25,88.43,86.07,223.0,-0.01,0.66,31.21,XR,NTT,DURABLES
15,BSOFT,836.99,-19.83,32.0,H-SC,13.82,90028.0,-50625.0,123401.0,-2.24,-35.99,137.07,51.74,131.0,-0.41,0.66,2.66,XR,ATH,IT
26,GREENPANEL,537.00,195.22,40.0,M-SC,1.00,131760.0,-51318.0,130297.0,-2.17,-28.03,98.89,43.14,230.0,-0.39,0.96,23.94,XY24,NTT,MISC
38,INDIGOPNTS,1408.00,102.57,31.0,M-SC,1.51,132816.0,-41743.0,41771.0,-1.79,-23.91,31.45,0.02,221.0,-1.00,0.97,15.53,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,10.45,124686.0,-17793.0,37319.0,4.53,-12.49,29.93,13.70,163.0,-0.48,0.91,47.16,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,36.0,M-SC,2.45,83303.0,-17474.0,17569.0,0.15,-17.34,21.09,0.09,245.0,-0.99,0.61,9.44,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,53.0,H-SC,1.55,222300.0,-47367.0,83696.0,0.17,-17.56,37.65,13.47,138.0,-0.57,1.63,12.44,XY24,NTT,PAINTS
18,CERA,9475.0,-22.20,38.0,H-SC,1.68,141714.0,-34189.0,76214.0,-0.29,-19.44,53.78,23.89,149.0,-0.45,1.04,22.39,OX40N,NTT,CERAMICS
66,SIS,528.0,2083.94,41.0,H-SC,4.02,88128.0,-22904.0,46514.0,0.12,-20.63,52.78,21.26,156.0,-0.49,0.64,19.07,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,55.0,M-MC,10.73,228578.0,3616.0,164028.0,1.31,1.61,71.76,74.53,192.0,0.02,1.67,34.38,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.78,47.0,H-MC,3.63,189390.0,24720.0,18617.0,0.20,15.01,9.83,26.31,89.0,1.33,1.39,24.19,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,47.0,H-MC,2.88,205650.0,13908.0,23855.0,2.30,7.25,11.60,19.69,99.0,0.58,1.50,14.54,XY25,NTT,AC
17,CAMS,5211.76,-6.05,43.0,H-SC,1.80,106700.0,4696.0,39234.0,0.28,4.60,36.77,43.06,122.0,0.12,0.78,22.51,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,54.0,M-LC,7.04,155144.0,14304.0,30191.0,1.77,10.16,19.46,31.59,66.0,0.47,1.13,39.33,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,53.0,H-MC,11.90,108857.0,8353.0,20759.0,2.40,8.31,19.07,28.96,79.0,0.40,0.80,45.10,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,57.0,H-MC,8.48,183683.0,3875.0,108538.0,2.66,2.16,59.09,62.52,88.0,0.04,1.34,36.98,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.01,25.0,H-SC,9.99,125350.0,2014.0,131743.0,-0.07,1.63,105.10,108.45,119.0,0.02,0.92,24.35,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-46.60,26.0,M-SC,1.31,90352.0,-1146.0,79898.0,-2.26,-1.25,88.43,86.07,223.0,-0.01,0.66,31.21,XR,NTT,DURABLES
86,WIPRO,420.00,-15.61,38.0,M-LC,5.46,148403.0,-2542.0,112000.0,1.53,-1.68,75.47,72.51,53.0,-0.02,1.09,4.24,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-6.17,29.0,H-LC,13.20,255033.0,-12332.0,77785.0,0.24,-4.61,30.50,24.48,48.0,-0.16,1.87,47.82,XY24,BTT,FINANCE
74,TCS,4476.75,-31.58,29.0,H-LC,12.18,272713.0,-73243.0,148110.0,0.07,-21.17,54.31,21.64,1.0,-0.49,1.99,0.07,X40,ATH,IT
5,ASIANPAINT,3465.66,-17.90,30.0,H-LC,6.54,207108.0,-44660.0,97879.0,0.46,-17.74,47.26,21.13,27.0,-0.46,1.51,11.40,X40,ATH,PAINTS
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.02,218246.0,-23678.0,86949.0,-0.37,-9.79,39.84,26.15,8.0,-0.27,1.60,3.13,X40,ATH,IT
83,VBL,671.64,-20.34,34.0,H-LC,8.49,287100.0,-28742.0,141397.0,1.18,-9.10,49.25,35.67,5.0,-0.20,2.10,3.49,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.94,43.0,H-LC,1.60,230918.0,-15233.0,94445.0,1.11,-6.19,40.90,32.18,16.0,-0.16,1.69,27.62,X200,ATH,IT
42,ITC,452.0,-38.77,45.0,H-LC,2.15,198094.0,-2044.0,22939.0,1.25,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.47,41.0,H-LC,2.41,155015.0,-31080.0,78562.0,0.13,-16.70,50.68,25.51,15.0,-0.40,1.13,14.23,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.41,43.0,H-LC,3.93,214656.0,4290.0,24492.0,-0.12,2.04,11.41,13.68,37.0,0.18,1.57,18.90,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-23.55,35.0,H-LC,4.24,204390.0,-110.0,41532.0,-0.50,-0.05,20.32,20.25,10.0,-0.00,1.49,8.67,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.18,272713.0,-73243.0,148110.0,0.07,-21.17,54.31,21.64,1.0,-0.49,1.99,0.07,X40,ATH,IT
80,UNITDSPR,1644.00,-14.23,45.0,H-LC,7.20,224959.0,-9185.0,57814.0,0.81,-3.92,25.70,20.77,86.0,-0.16,1.65,2.72,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.02,218246.0,-23678.0,86949.0,-0.37,-9.79,39.84,26.15,8.0,-0.27,1.60,3.13,X40,ATH,IT
83,VBL,671.64,-20.34,34.0,H-LC,8.49,287100.0,-28742.0,141397.0,1.18,-9.10,49.25,35.67,5.0,-0.20,2.10,3.49,X40N,ATH,FMCG
42,ITC,452.00,-38.77,45.0,H-LC,2.15,198094.0,-2044.0,22939.0,1.25,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.47,41.0,H-LC,2.41,155015.0,-31080.0,78562.0,0.13,-16.70,50.68,25.51,15.0,-0.40,1.13,14.23,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,45.0,H-LC,2.15,198094.0,-2044.0,22939.0,1.25,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG
19,CIPLA,1795.00,-23.55,35.0,H-LC,4.24,204390.0,-110.0,41532.0,-0.50,-0.05,20.32,20.25,10.0,-0.00,1.49,8.67,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.90,30.0,H-LC,6.54,207108.0,-44660.0,97879.0,0.46,-17.74,47.26,21.13,27.0,-0.46,1.51,11.40,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,43.0,H-LC,3.93,214656.0,4290.0,24492.0,-0.12,2.04,11.41,13.68,37.0,0.18,1.57,18.90,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.18,272713.0,-73243.0,148110.0,0.07,-21.17,54.31,21.64,1.0,-0.49,1.99,0.07,X40,ATH,IT
40,INFY,2275.00,-20.67,40.0,H-LC,8.19,308999.0,-3889.0,175573.0,0.12,-1.24,56.82,54.87,3.0,-0.02,2.26,5.32,X40,BTT,IT
42,ITC,452.00,-38.77,45.0,H-LC,2.15,198094.0,-2044.0,22939.0,1.25,-1.02,11.58,10.44,4.0,-0.09,1.45,4.49,X40,NTT,FMCG
83,VBL,671.64,-20.34,34.0,H-LC,8.49,287100.0,-28742.0,141397.0,1.18,-9.10,49.25,35.67,5.0,-0.20,2.10,3.49,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,47.0,H-LC,7.67,250382.0,-11237.0,130449.0,0.83,-4.30,52.10,45.57,7.0,-0.09,1.83,6.54,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,40.35,79680.0,-33869.0,73919.0,2.65,-29.83,92.77,35.27,268.0,-0.46,0.58,95.02,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,34.0,H-SC,12.27,89700.0,-8280.0,29879.0,-1.21,-8.45,33.31,22.05,152.0,-0.28,0.66,30.39,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-180.68,74.0,M-SC,6.74,92964.0,-9246.0,102632.0,12.42,-9.05,110.40,91.37,208.0,-0.09,0.68,50.91,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-45.32,53.0,H-MC,11.90,108857.0,8353.0,20759.0,2.40,8.31,19.07,28.96,79.0,0.40,0.80,45.10,MH,ATH,METALS
78,UJJIVANSFB,60.00,97.20,56.0,H-SC,10.45,124686.0,-17793.0,37319.0,4.53,-12.49,29.93,13.70,163.0,-0.48,0.91,47.16,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1164.75,63.0,M-SC,13.90,172044.0,-9099.0,54572.0,0.46,-5.02,31.72,25.10,235.0,-0.17,1.26,37.88,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-180.68,74.0,M-SC,6.74,92964.0,-9246.0,102632.0,12.42,-9.05,110.40,91.37,208.0,-0.09,0.68,50.91,XY25,NTT,FINANCE
60,SAIL,228.0,45.15,55.0,M-MC,10.73,228578.0,3616.0,164028.0,1.31,1.61,71.76,74.53,192.0,0.02,1.67,34.38,XY24,BTT,STEEL
81,VAIBHAVGBL,521.0,48.76,45.0,H-SC,5.79,128930.0,-53845.0,164914.0,-1.67,-29.46,127.91,60.77,125.0,-0.33,0.94,15.48,XR,NTT,JEWELLERY
7,ATULAUTO,844.0,3510.81,46.0,M-SC,7.40,112548.0,-34820.0,86639.0,-0.21,-23.63,76.98,35.16,236.0,-0.40,0.82,14.96,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.24
1,25,43.67
2,50,74.95


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.77
LC    32.50
MC    23.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.30
X40      14.77
X40N     11.44
XR       11.09
XY25     10.24
AR        8.26
OX40N     7.88
X200      1.69
MH        1.62
X5K       1.48
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.29
H-LC    25.95
H-MC    20.76
M-SC    13.02
M-LC     5.49
M-MC     2.65
L-SC     1.46
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.10,-6.44,41.58
IT,12.78,-20.28,85.87
FINANCE,10.10,-18.86,70.71
MISC,7.01,-19.89,86.54
BANKS,6.26,-16.48,77.75
PAINTS,5.64,-19.73,37.57
ELECTRICAL,5.46,-10.05,48.66
INSURANCE,3.76,-7.79,48.09
AC,3.60,1.56,14.93


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3236280.0,22
XR,1393099.0,14
AR,1201698.0,9
X40,828932.0,10
X40N,641364.0,9
OX40N,569028.0,10
XY25,439251.0,7
SR,254940.0,2
X5K,115950.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3787281.0,29
M-SC,1464416.0,17
H-LC,1271386.0,15
H-MC,1267858.0,15
M-LC,399716.0,4
M-MC,315612.0,2
L-SC,261411.0,3
L-MC,60892.0,1
L-LC,39876.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1250378.0      6
M-SC       XY24       840961.0      7
H-SC       AR         821382.0      5
           XR         811195.0      7
H-MC       XY24       579778.0      4
H-LC       X40        531450.0      5
M-MC       XY24       315612.0      2
H-SC       X40N       314178.0      4
           OX40N      262506.0      4
           SR         254940.0      2
H-LC       X40N       240743.0      3
H-MC       X40        220447.0      4
H-LC       AR         209011.0      2
H-MC       XY25       178791.0      2
L-SC       XR         172210.0      2
M-LC       XY24       171766.0      2
M-SC       AR         171305.0      2
           OX40N      144219.0      4
           XR         128264.0      2
H-LC       XY25       117952.0      3
M-LC       X5K        115950.0      1
           XR         112000.0      1
H-MC       XR         108538.0      1
M-SC       XY25       102632.0      1
H-LC       X200        94445.0      1
L-SC       OX40N       89201.0      1
H-MC       X40N        86443.0      2
H-LC       XY24        77785.0      1
M-SC       X40         77035.0      1
H-MC       OX40N       73102.0      1
H-SC       MH          72702.0      1
L-MC       XR          60892.0      1
L-LC       XY25        39876.0      1
H-MC       MH          20759.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
